## α-β-Pruning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-heuristic.ipynb

Die `value_AB()` Funktion ist ein Wrapper für die eigentliche Implementierung des α-β-Pruning in der Funktion `alphaBeta()`. Dieser Wrapper stellt Caching der Ergebnisse zur Verfügung, bzw. berechnet diese neu, falls der Cache keine oder invalide Ergenisse beinhaltet.

Ein Ergebnis aus dem Cache ist valide, solange das Intervall `alpha` und `beta` aus den Parametern innerhalb des im Cache verwendeten Intervalls `a` und `b` liegt. Also das Intervall des Caches muss genereller sein, als das Intervall aus den Paramtern.

In [ ]:
Cache = {}

def value_AB(s, p, limit, alpha=-1, beta=1):
    global Cache
    if s in Cache:
        (val, a, b) = Cache[s]
        if a <= alpha and beta <= b:
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = alphaBeta(s, p, limit, alpha, beta)
            Cache[s] = (val, alpha, beta)
            return val
    else:
        val = alphaBeta(s, p, limit, alpha, beta)
        Cache[s] = (val, alpha, beta)
        return val

Die Funktion `alphaBeta()` beinhaltet nun die eigentliche Implementierung des α-β-Pruning.

* Wie zuvor beim Minimax Algorithmus, wird der `utility` Wert zurückgegeben, falls das Spiel in dem State `s` beendet (`finished`) ist.
* Ebenfalls äquivalent wird der `heuristic` Wert verwendet, sobald das Rekursionslimit (`limit`) erreicht wird.
* Der eigentliche α-β-Pruning Alogrithmus errechnet rekursiv mit Hilfe des Caches (`value_AB()`) den Wert eines Zuges. Hierbei wird der erste Wert der nächsten States verwendet, der größer oder gleich der oberen Grenze `beta` ist.

In [ ]:
def alphaBeta(s, p, limit, alpha, beta):
    nss = nextStates(s, p)
    if finished(s, p, ns=nss):
        return utility(s, p)
    if limit == 0:
        return heuristic(s, p)
    val = alpha
    for ns in nss:
        val = max(val, -value_AB(ns, opponent(p), limit-1, -beta, -alpha))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

`bestMove_AB()` wählt mit Hilfe des α-β-Pruning Algorithmuses den besten State aus allen möglichen nächsten States aus. Hierzu werden die Werte aller States errechnet und der State mit dem höchsten Wert ausgewählt.

In [ ]:
def bestMove_AB(s, p):
    # Clear cache
    global Cache
    Cache = {}

    moves = [
        (-value_AB(s, opponent(p), 3), s)
        for s in nextStates(s, p)
    ]
    return max(moves, key=lambda m: m[0])